In [1]:
import numpy as np
import pandas as pd
import time
import random
import math
from math import log

from sklearn import neighbors, datasets
from sklearn.neighbors import KNeighborsClassifier

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

### Load and Prepare Data

In [11]:
dataframe = pd.read_csv('./data.csv')

data_size = dataframe.shape[0]
feature_num = dataframe.shape[1] - 1
test_size = round(data_size * 0.2)
#print(test_size)

# normalization
df = dataframe.iloc[:, 0:48]
xmax = df.max()
xmin = df.min()

dff=(df-xmin)/(xmax-xmin)
#print(dff)

dataframee = pd.concat([dff,dataframe['48']],axis=1)

#Split data
random.seed(0)
indices = dataframee.index.tolist()
#print(indices)
test_in = random.sample(population = indices, k = test_size)
#print(test_in)

test_df = dataframee.loc[test_in]
train_df = dataframee.drop(test_in)



X_train_df = (train_df.iloc[:, 0:48])
Y_train_df = train_df['48']

X_test_df = (test_df.iloc[:, 0:48])
Y_test_df = test_df['48'] 

X_train = X_train_df.to_numpy()
Y_train = Y_train_df.to_numpy()
X_test = X_test_df.to_numpy()
Y_test = Y_test_df.to_numpy()

print(X_train.shape,Y_train.shape)
    
print("size of training data:",len(X_train))
print("size of testing data:", len(X_test))



(32765, 48) (32765,)
size of training data: 32765
size of testing data: 8191


In [3]:
dataframee.head()

,0,1,2,3,4,5,6,7,8,9,...,39,40,41,42,43,44,45,46,47,48
0,0.596225,0.540284,0.615606,0.838338,0.717266,0.773620,0.710818,0.710995,0.712087,0.644000,...,0.000151,0.012935,0.110421,0.320896,0.364686,0.358736,0.317241,0.344491,0.308625,5
1,0.596119,0.541987,0.657480,0.839107,0.714800,0.756745,0.784912,0.785015,0.784224,0.710168,...,0.000240,0.015774,0.058570,0.464179,0.524752,0.552045,0.228966,0.256625,0.221024,8
2,0.596369,0.544879,0.644498,0.839618,0.724341,0.781678,0.688424,0.688369,0.688078,0.533995,...,0.000351,0.038273,0.148641,0.383582,0.435644,0.444238,0.329655,0.358438,0.320755,9
3,0.596565,0.543414,0.630369,0.838824,0.702555,0.798007,0.700639,0.700660,0.701047,0.516568,...,0.000167,0.006927,0.048715,0.326866,0.372937,0.379182,0.371034,0.400279,0.362534,9
4,0.596308,0.545396,0.646339,0.838603,0.722751,0.783740,0.764063,0.763992,0.763704,0.582751,...,0.000350,0.028285,0.112182,0.391045,0.443894,0.453532,0.325517,0.354254,0.316712,6


### KNN

In [4]:
def KNN_find_similarity(X_train, X_test):
    norm_test = np.linalg.norm(X_test, axis = 1)
    norm_test = np.reshape(norm_test,(-1,1))
    norm_train = np.linalg.norm((X_train), axis = 1)
    norm_train = np.reshape(norm_train,(1,-1))
    
    norm = np.dot(norm_test,norm_train) 
    cos_sim = np.dot(X_test, np.transpose(X_train))/norm
    
    return cos_sim

def KNN_predict(cos_sim,K,Y_train):
    Y_pred = []
    for row in cos_sim:
        votes=[]
        index = row.argsort()[::-1][0:K]
        for i in index:
            votes.append(Y_train[i])
        mmax = np.argmax(np.bincount(votes))
        Y_pred.append(mmax)
    Y_pred = np.asarray(Y_pred)
    return Y_pred
    

## K-Means

In [5]:
#find k cluster center
#use test set to find which cluster center it similar to 
def update_centroids(centers, X_train):
    #k = len(centers)
    k = centers.shape[0]
    n = X_train.shape[0]
    d = X_train.shape[1]
    
    #centers = np.asarray(centers)
    #print(centers.shape)
    #print(X_train.shape)
    #print("AAAAAAAAAAAAAAAAAAAA")
    
    #compute l2
    aa = np.sum(np.square(centers),axis = 1)
    bb = np.sum(np.square(X_train),axis = 1)
    #print(aa.shape)
    #print(bb.shape)
    aa = np.reshape(aa,(-1,1))
    bb = np.reshape(bb,(-1,1))
    #print(aa.shape)
    #print(bb.shape)
    aa = np.tile(aa,n)
    bb = np.tile(bb,k)
    bb = np.transpose(bb)
    
    cc = -2 * np.dot(centers,np.transpose(X_train))
    #print(aa.shape)
    #print(bb.shape)
    #print(cc.shape)
    #print("BBBBBBBBBBBBBBBBBBBBBB")
    xx = aa + bb + cc
    #print("bbbbbbbbbbbbbbbbbbbbbbbb")
    dist = np.sqrt(xx)
    #print("bbbbbbbbbbccccccccccccccc")
    colmax = dist.argmax(axis=0)
    #print("CCCCCCCCCCCCCCCCCCCCCCCC")
    
    
    #new_centers = [np.zeros(d)] * k
    #new_centers = np.asarray(new_centers)
    
    #print("new_centers",new_centers.shape)
    
    new_centers = np.zeros((k,d))
    
    clusters = [np.zeros((n,d))] * k
    
    
    for i in range(n):
        cluster_i = colmax[i]
        #clusters[cluster_i].append(X_train[i])
        clusters[cluster_i][i,:]=X_train[i]
    #print("DDDDDDDDDDDDDDDDDDDDD")
    
    
    #clusters = np.asarray(clusters)
    for j in range(k):
        new_centers[j] = np.mean(clusters[j],axis=0)
    #print("EEEEEEEEEEEEEEEEEEEEEEEEE")
    #print(type(new_centers))
    
    return new_centers  # k * d


In [6]:
def find_centroids(X_train,K):
    d = X_train.shape[1]
    
    #create a matrix for centers
    final_centers = [np.zeros(d)] * K
    #centers = np.asarray(centers)
    centers = np.zeros((K,d))
    #print(centers.shape)
    #print(X_train.shape)
    
    #if center still change after 10000 times, force stop
    itera = 10000
    while(itera!=0):
        print(itera)
        
        #initial k centers
        if itera == 10000:
            #print("111111111111")
            ind = random.sample(range(0,len_train),K)
            #print(ind)
            ii = 0
            for i in ind:
                centers[ii,:] = X_train[i,:]
                ii = ii + 1
                #print(X_train[i].shape)
                #centers[i] = X_train[i]
        #print("2222222222222")
        
        #update centers  
        new_centers = update_centroids(centers, X_train)
        #print("333333333333333333")
        
        print(type(new_centers),type(centers),new_centers.shape)       
        if ((new_centers == centers).all()):
            #print("endendendendendend")
            break
        #count = 0
        #for i in range(K):
        #    if ((new_centers[i]==centers[i]).all()):
        #        count = count + 1
        #if count == K:
        #    break
        #print("4444444444444444")
        centers = new_centers
        itera = itera - 1
        
    final_centers = new_centers
    
    return final_centers
    #return new_centers

## PCA

In [7]:
#X_train, N

# mean of feature vectors
mean_vector = np.mean(X_train,axis = 0)

# covariance matrix

## Random Forest

In [21]:
class node:
    '''树的节点的类
    '''
    def __init__(self, fea=-1, value=None, results=None, right=None, left=None):
        self.fea = fea  # 用于切分数据集的属性的列索引值
        self.value = value  # 设置划分的值
        self.results = results  # 存储叶节点所属的类别
        self.right = right  # 右子树
        self.left = left  # 左子树

In [118]:
def build_tree(data):
    '''构建树
    input:  data(list):训练样本
    output: node:树的根结点
    '''
    # 构建决策树，函数返回该决策树的根节点
    if len(data) == 0:
        return node()
    # 1、计算当前的Gini指数
    currentGini = cal_gini_index(data)
    bestGain = 0.0
    bestCriteria = None  # 存储最佳切分属性以及最佳切分点
    bestSets = None  # 存储切分后的两个数据集
    feature_num = len(data[0]) - 1  # 样本中特征的个数
    # 2、找到最好的划分
    for fea in range(0, feature_num):
        # 2.1、取得fea特征处所有可能的取值
        feature_values = {}  # 在fea位置处可能的取值
        for sample in data:  # 对每一个样本
            feature_values[sample[fea]] = 1  # 存储特征fea处所有可能的取值
        # 2.2、针对每一个可能的取值，尝试将数据集划分，并计算Gini指数
        for value in feature_values.keys():  # 遍历该属性的所有切分点
            # 2.2.1、 根据fea特征中的值value将数据集划分成左右子树
            (set_1, set_2) = split_tree(data, fea, value)
            # 2.2.2、计算当前的Gini指数
            nowGini = float(len(set_1) * cal_gini_index(set_1) + 
                             len(set_2) * cal_gini_index(set_2)) / len(data)
            # 2.2.3、计算Gini指数的增加量
            gain = currentGini - nowGini
            # 2.2.4、判断此划分是否比当前的划分更好
            if gain > bestGain and len(set_1) > 0 and len(set_2) > 0:
                bestGain = gain
                bestCriteria = (fea, value)
                bestSets = (set_1, set_2)
    # 3、判断划分是否结束
    if bestGain > 0:
        right = build_tree(bestSets[0])
        left = build_tree(bestSets[1])
        return node(fea=bestCriteria[0], value=bestCriteria[1], 
                    right=right, left=left)
    else:
        return node(results=(label_uniq_cnt(data)).item(0))  # 返回当前的类别标签作为最终的类别标签

In [102]:
def cal_gini_index(data):
    '''计算给定数据集的Gini指数
    input:  data(list):树中
    output: gini(float):Gini指数
    '''
    total_sample = len(data)  # 样本的总个数
    if len(data) == 0:
        return 0  
    label_counts = label_uniq_cnt(data)  # 统计数据集中不同标签的个数
    # 计算数据集的Gini指数
    gini = 0
    for label in range (len(label_counts)):
        gini = gini + pow(label_counts[label], 2) 
    gini = 1 - float(gini) / pow(total_sample, 2)
    return gini

In [103]:
def split_tree(data, fea, value):
    '''根据特征fea中的值value将数据集data划分成左右子树
    input:  data(list):数据集
            fea(int):待分割特征的索引
            value(float):待分割的特征的具体值
    output: (set1,set2)(tuple):分割后的左右子树
    '''
    set_1 = []
    set_2 = []
    for x in data:
        if x[fea] >= value:
            set_1.append(x)
        else:
            set_2.append(x)
    return (set_1, set_2)

In [104]:
def predict(sample, tree):
    '''对每一个样本sample进行预测
    input:  sample(list):需要预测的样本
            tree(类):构建好的分类树
    output: tree.results:所属的类别
    '''
    # 1、只是树根
    if tree.results != None:
        return tree.results
    else:
    # 2、有左右子树
        val_sample = sample[tree.fea]
        branch = None
        if val_sample >= tree.value:
            branch = tree.right
        else:
            branch = tree.left
        return predict(sample, branch)

In [105]:
def random_forest_training(data_train, trees_num):
    '''构建随机森林
    input:  data_train(list):训练数据
            trees_num(int):分类树的个数
    output: trees_result(list):每一棵树的最好划分
            trees_feature(list):每一棵树中对原始特征的选择
    '''
    trees_result = []  # 构建好每一棵树的最好划分
    trees_feature = []
    n = np.shape(data_train)[1]  # 样本的维数
    if n > 2:
        k = int(log(n - 1, 2)) + 1 # 设置特征的个数
    else:
        k = 1
    # 开始构建每一棵树
    for i in range(trees_num):
        # 1、随机选择m个样本, k个特征
        data_samples, feature = choose_samples(data_train, k)
        # 2、构建每一棵分类树
        tree = build_tree(data_samples)
        # 3、保存训练好的分类树
        trees_result.append(tree)
        # 4、保存好该分类树使用到的特征
        trees_feature.append(feature)
   
    return trees_result, trees_feature

In [106]:
def choose_samples(data, k):
    '''
    input:  data(list):原始数据集
            k(int):选择特征的个数
    output: data_samples(list):被选择出来的样本
            feature(list):被选择的特征index
    '''
    m, n = np.shape(data)  # 样本的个数和样本特征的个数
    # 1、选择出k个特征的index
    feature = []
    for j in range(k):
        feature.append(random.randint(0, n - 2))  # n-1列是标签
    # 2、选择出m个样本的index
    index = []
    for i in range(m):
        index.append(random.randint(0, m - 1))
    # 3、从data中选择出m个样本的k个特征，组成数据集data_samples
    data_samples = []
    for i in range(m):
        data_tmp = []
        for fea in feature:
            data_tmp.append(data[index[i]][fea])
        data_tmp.append(data[index[i]][-1])
        data_samples.append(data_tmp)
    return data_samples, feature

In [138]:
def label_uniq_cnt(data):
    data = np.asarray(data)
    classes = data[:,-1]
#     print(classes)
    index_elements, counts_elements = np.unique(classes, return_counts=True)
    if len(counts_elements) > 1:
        return counts_elements
    else:
        return index_elements   

In [141]:
t = np.array([[1,1],[1,1],[2,1],[3,2],[4,3]])
t2 = np.array([[1,1],[1,1]])
label_uniq_cnt(t2)
# cal_gini_index(t2)

array([1])

In [109]:
# random_forest_training(data_train, trees_num)

In [161]:
X1 = X_train[0:1000, :]
Y1 = Y_train[0:1000]

Y1 = np.reshape(Y1,(-1, 1))
data_train = np.concatenate((X1,Y1),axis = 1)

trees_num = 2

# predict(X1,Y1, 2)
# random_forest_training(data_train, trees_num)



In [162]:
def rf(x_test, data_train, trees_num):
    trees, features = random_forest_training(data_train, trees_num)
    ans = []
    for tree in trees:
        ans.append(predict(x_test, tree))
    print("prediction is :" ,ans)
    return np.asarray(ans)
    

In [163]:
rf(X_test[0], data_train, 10)
print(Y_test[0])

prediction is : [11.0, 11.0, 11.0, 11.0, 10.0, 7.0, 11.0, 7.0, 10.0, 11.0]
5


## Confusion Matrix

In [25]:

def confusMax(Y_pred, Y_test):
    num_class = len(np.unique(Y_test))
#     print(np.unique(Y_test))
    Y_hold = Y_test * num_class + Y_pred
#     Y_hold = np.zeros(num_class * num_class)
    Y_bin = np.arange(num_class * num_class + 1)
    con_matrx = np.asarray(np.histogram(Y_hold, bins=Y_bin)[0]).reshape(num_class, num_class)
#     return np.histogram(Y_hold, bins=Y_bin)
    return con_matrx
#     print(len(hold))
    
    

In [26]:
# Y1 = KNN(X_train,X_test,Y_train,5)

confusMax(Y1, Y_test)

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0, 795,   0,   0,   0,   0,  12,   0,   0,   0,   0],
       [  0,   0, 747,   0,   0,   0,   0,   0,   0,   0,  11],
       [  0,   0,   0, 726,   0,   4,   1,   0,   1,   0,   0],
       [  0,   0,   0,   2, 721,   2,   0,   0,   0,   0,   0],
       [  0,   1,   0,   5,   1, 712,   1,   0,   6,   0,   0],
       [  1,  12,   0,   2,   0,   0, 711,   0,   1,   7,   0],
       [  0,   0,   0,   0,   0,   0,   0, 754,   0,   0,   0],
       [  0,   2,   0,   1,   2,   7,   5,   0, 700,   0,   0],
       [  0,   3,   0,   3,   0,   0,  22,   0,   0, 721,   0],
       [  0,   0,  26,   0,   0,   0,   0,   0,   0,   0, 704]])

## Recall

In [45]:
def Recall(y_true,y_pred):
    num_class = len(np.unique(Y_test))
    print(num_class)
    con_matrix = confusMax(y_pred,y_true)
    sum_recall = 0.0
    for j in range(num_class):
        TP = 0
        FN = 0
        for i in range(num_class):
            if(i == j):
                TP += con_matrix[i][j]
            else: 
                FN += con_matrix[i][j]
        if TP == 0:
            sum_recall += 0
        else:             
            sum_recall += (TP/(TP + FN))
    recall = sum_recall/num_class
    return recall
            

            
        
#      """
#     :type y_true: numpy.ndarray
#     :type y_pred: numpy.ndarray
#     :rtype: float
#     """

In [46]:
# num_class = len(np.unique(Y_test))

Recall(Y_test, Y1)

11


0.8921579871684698

In [47]:
def Precision(y_true,y_pred):
    num_class = len(np.unique(Y_test))
    con_matrix = confusMax(y_pred,y_true)
#     num = len(con_matrix)
#     print(num)
    sum_prec = 0.0
    for i in range(num_class):
        TP = 0
        FP = 0
        for j in range(num_class):
            if(i == j):
                TP += con_matrix[i][j]
            else: 
                FP += con_matrix[i][j]
        if TP == 0:
            sum_prec += 0
        else:             
            sum_prec += (TP/(TP + FP))
        prec = sum_prec/num_class
    return prec
            

In [48]:
Precision(Y_test, Y1)

0.8917796237059065

## Functions

In [10]:
# -*- coding: utf-8 -*-
"""
Predicitve_Analytics.py
"""


def Accuracy(y_true,y_pred):
    """
    :type y_true: numpy.ndarray
    :type y_pred: numpy.ndarray
    :rtype: float
    
    """
    l = len(y_true)
    count = 0
    for i in range(l):
        if y_true[i] == y_pred[i]:
            count = count + 1
    acc = count/l
    return acc

def Recall(y_true,y_pred):
     """
    :type y_true: numpy.ndarray
    :type y_pred: numpy.ndarray
    :rtype: float
    """

def Precision(y_true,y_pred):
    """
    :type y_true: numpy.ndarray
    :type y_pred: numpy.ndarray
    :rtype: float
    """
def WCSS(Clusters):
    """
    :Clusters List[numpy.ndarray]
    :rtype: float
    """
    
def ConfusionMatrix(y_true,y_pred):
    
    """
    :type y_true: numpy.ndarray
    :type y_pred: numpy.ndarray
    :rtype: float
    """  

def KNN(X_train,X_test,Y_train,N):
    """
    :type X_train: numpy.ndarray
    :type X_test: numpy.ndarray
    :type Y_train: numpy.ndarray
    :typr N: constant
    
    :rtype: numpy.ndarray
    """
    cos_sim = KNN_find_similarity(X_train, X_test)
    Y_pred = KNN_predict(cos_sim,N,Y_train)
    
    return Y_pred
        
def RandomForest(X_train,Y_train,X_test):
    """
    :type X_train: numpy.ndarray
    :type X_test: numpy.ndarray
    :type Y_train: numpy.ndarray
    
    :rtype: numpy.ndarray
    """
    
def PCA(X_train,N):
    """
    :type X_train: numpy.ndarray
    :type N: int
    :rtype: numpy.ndarray
    """
    
def Kmeans(X_train,N):
    """
    :type X_train: numpy.ndarray
    :type N: int
    :rtype: List[numpy.ndarray]
    """
    centroids = find_centroids(X_train,N)
    return centroids

def SklearnSupervisedLearning(X_train,Y_train,X_test):
    """
    :type X_train: numpy.ndarray
    :type X_test: numpy.ndarray
    :type Y_train: numpy.ndarray
    
    :rtype: List[numpy.ndarray] 
    """

def SklearnVotingClassifier(X_train,Y_train,X_test):
    
    """
    :type X_train: numpy.ndarray
    :type X_test: numpy.ndarray
    :type Y_train: numpy.ndarray
    
    :rtype: List[numpy.ndarray] 
    """


#"""
#Create your own custom functions for Matplotlib visualization of hyperparameter search. 
#Make sure that plots are labeled and proper legends are used
#"""

## Main()

In [36]:
len_train = len(Y_train)
len_test = len(Y_test)

## KNN
K_knn = 11

s1 = time.time()
Y_pred_knn = KNN(X_train,X_test,Y_train,K_knn)
e1 = time.time()

#Y_pred_knn = np.asarray(Y_pred_knn)
#print(Y_pred_knn.shape)
#print(Y_test.shape)
#acc_knn = Accuracy(Y_test, Y_pred_knn)
#print(acc_knn)
acc_knn = Accuracy(Y_test,Y_pred_knn)

print("Running time of KNN",e1-s1,".")
print("Accuracy of KNN",acc_knn,".")
print("============================")

## K-Means
K_kmeans = 11

s2 = time.time()
centroids = Kmeans(X_train,K_kmeans)
e2 = time.time()
print("Runtime of K-Means: ",e2-s2,"s.")
#print(centroids)

## PCA



Running time of KNN 33.30502200126648 .
Accuracy of KNN 0.9841289219875473 .
10000
<class 'numpy.ndarray'> <class 'numpy.ndarray'> (11, 48)
9999
<class 'numpy.ndarray'> <class 'numpy.ndarray'> (11, 48)
Runtime of K-Means:  0.1835639476776123 s.


/Users/yuxuanzhang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in sqrt


In [ ]:
a = [np.array([1,2,3,4])]*5
print(a)
b = np.asarray(a)
print(b)
print(b.shape)
aa = np.sum(np.square(a),axis = 1)
print(aa)

In [2]:
import numpy as np
a = np.zeros((3,2))
print(a)

[[0. 0.]
 [0. 0.]
 [0. 0.]]


In [3]:
a = np.array([[1,2,3,4],[2,3,4,5],[3,4,5,6],[4,5,6,7],[5,6,7,8]])
print(a.shape)
x = np.zeros((6,4))
x[1,:] = a[0]
print(x)


#print(a[:,0])
#print(a.max(axis=0))
b = a.argmax(axis=0)
#print(b[0])

aaa = []*11
aaa.append([1,2,3])
aaa.append([222])
aaa.append([3,2,1])
#print(aaa[0])

(5, 4)
[[0. 0. 0. 0.]
 [1. 2. 3. 4.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [29]:
from sklearn.cluster import KMeans
    
kmeans = KMeans(n_clusters=11).fit(train_df)
centroids = kmeans.cluster_centers_
print(centroids)



[[5.96222509e-01 5.46346345e-01 6.46898682e-01 8.38899954e-01
  7.27504165e-01 7.83784611e-01 5.29478346e-01 5.29326875e-01
  5.28742346e-01 5.47212657e-01 5.47290494e-01 5.47474042e-01
  1.58367340e-02 2.03587274e-02 6.42754199e-02 1.57654043e-02
  3.56626558e-02 4.78043144e-02 5.32182908e-01 5.32177568e-01
  5.32247871e-01 5.33635662e-01 5.33666652e-01 5.33752106e-01
  5.41685345e-01 5.62270157e-01 4.52617717e-01 5.43255924e-01
  5.14683908e-01 5.86250793e-01 3.22484271e-01 3.37697482e-01
  3.28006638e-01 1.94725347e-01 1.97781648e-01 2.00199773e-01
  5.70687900e-04 3.75546138e-02 6.60251842e-02 3.49249768e-04
  4.87408056e-02 7.45192054e-02 3.72725076e-01 4.23218629e-01
  4.32447952e-01 3.24090481e-01 3.52505404e-01 3.16465685e-01
  2.00000000e+00]
 [5.96065939e-01 5.43367815e-01 6.41493724e-01 8.38773637e-01
  7.16552764e-01 7.81160565e-01 7.01563656e-01 7.01586333e-01
  7.01451618e-01 5.42056622e-01 5.42226998e-01 5.42502400e-01
  1.67388036e-02 2.00252471e-02 6.43814050e-02 1.642

In [ ]:
knn = KNeighborsClassifier(n_neighbors=11)
knn.fit(X_train,Y_train)
a = Accuracy(Y_test,knn.predict(X_test))
print(a,"==========11")